# Decorator
A decorator is a function that takes another function and extends the behavior of this function without explicitly modifying it. It is a very powerful tool that allows to add new functionality to an existing function.

There are 2 kinds of decorators:
* Function decoratos
* Class decorators

A function is decorated with the @ symbol


## Function decorators
Functions in Python are first class objects, which means that – like any other object – they can be defined inside another function, passed as argument to another function, or returned from other functions.
A decorator is a function that takes another function as argument, wraps its behaviour inside an inner function. and returns the wrapped function.

This is the normal method

In [1]:
# A decorator function takes another function as argument, wraps its behaviour inside
# an inner function, and returns the wrapped function.
def start_end_decorator(func):
    def wrapper():
        print('Start')
        func()
        print('End')
    return wrapper

def print_name():
    print('Alex')

# Now wrap the function by passing it as argument to the decorator function
# and asign it to itself -> Our function has extended behaviour!
print_name_with_wrapper=start_end_decorator(print_name)
print_name_with_wrapper()

Start
Alex
End


## The decorator syntax
Instead of wrapping our function and asigning it to itself, we can achieve the same thing simply by decorating our function with an @.

In [2]:
def start_end_decorator(func):
    def wrapper(x):
        print('Start')
        func(x)
        print('End')
    return wrapper

@start_end_decorator
def print_name(name):
    print(name)

input_name='Alex'
print_name(input_name)


Start
Alex
End


## With function arguments
If our function has input arguments and we try to wrap it with our decorator above, it will raise a TypeError since we have to call our function inside the wrapper with this arguments, too. However, we can fix this by using *args and **kwargs in the inner function.

In [3]:
def start_end_decorator_2(func):
    #  using *args and **kwargs in the inner function
    def wrapper(*args, **kwargs):
        print('Start')
        func(*args, **kwargs)
        print('End')
    return wrapper

@start_end_decorator_2
def add_5(x):
    return x + 5

result = add_5(10)
print(result)

Start
End
None


## Return Values
Note that in the example above, we do not get the result back, so as next step we also have to return the value from our inner function.

In [4]:
def start_end_decorator_3(func):
    
    def wrapper(*args, **kwargs):
        print('Start')
        result = func(*args, **kwargs)
        print('End')
        return result
    return wrapper

@start_end_decorator_3
def add_5(x):
    return x + 5

result = add_5(10)
print(result)

Start
End
15


## Function identity

If we have a look at the name of our decorated function, and inspect it with the built-in help function, we notice that Python thinks our function is now the wrapped inner function of the decorator function.

In [5]:
print(add_5.__name__)
help(add_5)


wrapper
Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



To fix this, use the functools.wraps decorator, which will preserve the information about the original function. This is helpful for introspection purposes, i.e. the ability of an object to know about its own attributes at runtime:

In [6]:
import functools
def start_end_decorator_4(func):
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('Start')
        result = func(*args, **kwargs)
        print('End')
        return result
    return wrapper

@start_end_decorator_4
def add_5(x):
    return x + 5
result = add_5(10)
print(result)
print(add_5.__name__)
help(add_5)

Start
End
15
add_5
Help on function add_5 in module __main__:

add_5(x)



## Decorator function arguments

Decorator with input argument

In [7]:
def repeat(num_times):
    def decorator_repeat(func):
        def wrapper(*args,**kwargs):
            for _ in range(num_times):
                result=func(*args,**kwargs)
            return result
        return wrapper
    return decorator_repeat

@repeat(num_times=4)
def greet(name):
    print(f'Hello {name}')

greet('Pete')


Hello Pete
Hello Pete
Hello Pete
Hello Pete


## The final template for own decorators
Now that we have all parts, our template for any decorator looks like this.

In [8]:
import functools

def my_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Do something before
        result = func(*args, **kwargs)
        # Do something after
        return result
    return wrapper

## Nest Decorators
We can apply several decorators to a function by stacking them on top of each other. The decorators are being executed in the order they are listed.

In [9]:
# a decorator function that prints debug information about the wrapped function
def debug(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]
        signature = ", ".join(args_repr + kwargs_repr)
        print(f"Calling {func.__name__}({signature})")
        result = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {result!r}")
        return result
    return wrapper

# a decorator that prints "start" and "end"
def start_end_decorator_4(func):
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('Start')
        result = func(*args, **kwargs)
        print('End')
        return result
    return wrapper

@debug
@start_end_decorator_4
def say_hello(name):
    greeting = f'Hello {name}'
    print(greeting)
    return greeting

# now `debug` is executed first and calls `@start_end_decorator_4`, which then calls `say_hello`
say_hello(name='Alex')

Calling say_hello(name='Alex')
Start
Hello Alex
End
'say_hello' returned 'Hello Alex'


'Hello Alex'

## Class decorator 
We can also use a class as a decorator. Therefore, we have to implement the "__call__()" method to make our object callable. Class decorators are typically used to maintain a state, e.g. here we keep track of the number of times our function is executed. The "__call__()" method does essentially the same thing as the wrapper() method we have seen earlier.

In [10]:
class CountCalls:
    # the init needs to have the func as argument and stores it
    def __init__(self,func):
        self.func=func
        self.num_calls=0

    # extend functionality, execute function, and return the result
    def __call__(self,*args,**kwargs):
        self.num_calls+=1
        print(f'This is executed {self.num_calls} times')
        return self.func(*args,**kwargs)

@CountCalls
def say_hello():
    print('Hello')

say_hello()
say_hello()

This is executed 1 times
Hello
This is executed 2 times
Hello


## Some typical use cases
* Use a timer decorator to calculate the execution time of a function
* Use a debug decorator to print out some more information about the called function and its arguments
* Use a check decorator to check if the arguments fulfill some requirements and adapt the bevaviour accordingly
* Register functions (plugins)
* Slow down code with time.sleep() to check network behaviour
* Cache the return values for memoization (https://en.wikipedia.org/wiki/Memoization)
* Add information or update a state